# National Park Service MCP Server Integration workbook

In [7]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio
from IPython.display import display, Markdown

load_dotenv(override=True)

ModuleNotFoundError: No module named 'agents'